<a href="https://colab.research.google.com/github/pinholuc/mlops/blob/master/ml_handbook/5.%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Regressao Logistica

"""
Estima probabilidades usando uma funcao logistica.

1. Eficiencia na execucao: pode usar n_jobs se nao estiver com solucionador
'liblinear'

2. Pre-processamento de dados:

Se solver estiver definido como 'sag' ou 'saga', padronize para que haja
convergencia. Eh capaz de lidar com entradas esparsas.

3. Evitar overfitting: o parametro C controla a regularizacao. Eh possivel
especificar penalty com 'l1' ou 'l2'

4. Interpretacao dos resultados: o atributo .coef do modelo apos a adequacao
mostra os coeficientes da funcao de decisao. uma mudanca de x em uma unidade 
modifica o log odds ratio de acordo com o coeficiente. O atributo .intercept_
eh o inverso do log odds da condicao de base
"""



"\nEstima probabilidades usando uma funcao logistica.\n\n1. Eficiencia na execucao: pode usar n_jobs se nao estiver com solucionador\n'liblinear'\n\n2. Pre-processamento de dados:\n\nSe solver estiver definido como 'sag' ou 'saga', padronize para que haja\nconvergencia. Eh capaz de lidar com entradas esparsas.\n\n3. Evitar overfitting: o parametro C controla a regularizacao. Eh possivel\nespecificar penalty com 'l1' ou 'l2'\n\n4. Interpretacao dos resultados: o atributo .coef do modelo apos a adequacao\nmostra os coeficientes da funcao de decisao. uma mudanca de x em uma unidade \nmodifica o log odds ratio de acordo com o coeficiente. O atributo .intercept_\neh o inverso do log odds da condicao de base\n"

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

df = pd.read_excel("./titanic3.xls").drop(columns = ["name", 
                                                      "ticket", 
                                                      "home.dest", 
                                                      "boat", 
                                                      "body", 
                                                      "cabin"])

# Getting rid of string/object features
df = pd.get_dummies(df).drop(columns="sex_male").dropna()

# Segregate features (x) and labels (y)
X = df.drop(columns="survived")

# Labels (y)
y = df.survived

# Split in train and validation
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)


In [3]:
lr = LogisticRegression(random_state=42, max_iter=2000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=2000, random_state=42)

In [4]:
lr.score(X_test, y_test)
lr.predict(X.iloc[[0]])
lr.predict_proba(X.iloc[[0]])
lr.predict_log_proba(X.iloc[[0]])
lr.decision_function(X.iloc[[0]])

array([2.77932032])

In [5]:
"""

o logit inverso dos coeficientes nos da a proporcao de casos positivos.
Nesse caso, se sex for female, ha mais chances de sobrevi-
vencia.

"""

def inv_logit(p):
  return np.exp(p) / (1 + np.exp(p))

cols = X.columns
for col, val in sorted(
    zip(cols, lr.coef_[0]),
    key=lambda x: x[1],
    reverse=True
):
  print(
      f"{col:10}{val:10.3f} {inv_logit(val):10.3f}"
  )

sex_female     2.719      0.938
embarked_C     0.456      0.612
embarked_S     0.042      0.510
parch          0.040      0.510
fare           0.001      0.500
age           -0.043      0.489
sibsp         -0.384      0.405
embarked_Q    -0.580      0.359
pclass        -1.040      0.261


In [7]:
# Naive Bayes

"""
Eh um classificador probabilistico que pressupoe uma independencia
entre os atributos dos dados. Eh popular para aplicacoes de classificacao de textos
como por exemplo identificacao de spams.

Uma vantagem desse modelo eh que, por supor uma independencia entre os atributos,
ele eh capaz de fazer o treinamento de um modelo com numero pequeno de amostras

Ha 3 classes para Naive Bayes no scikit-learn: GaussianNB, MultinomialNB
e BernoulliNB.

1. Eficienca na execucao: treinamento O(Nd)
"""

from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)
nb.score(X_test, y_test)


0.7579617834394905

In [8]:
nb.predict(X.iloc[[0]])

array([1])

In [11]:
# Support Vector Machines (SVM)


"""
Algoritmo que tenta fazer adequacacao de uma linha (ou plano ou hiperplano)
entre as diferentes classes de modo a maximizar a distancia da linha 
ate os pontos das classes.

Dessa maneira, ela tenta encontrar uma separacao robusta entre as classes. Os
vetores suport (support vectors) sao os pontos da fronteira do hiperplano
divisor.

A implementacao do scikit-learn eh O(n4) para o treinamento portanto pode ser
dificil escalar para tamanhos maiores. 

O algoritmo nao eh invariante a escala. Padronizar os dados eh extremamente
recomendavel.
"""

from sklearn.svm import SVC

svc = SVC(random_state=42, probability=True)
svc.fit(X_train, y_train)
svc.score(X_test, y_test)



0.6592356687898089

In [12]:
svc.predict(X.iloc[[0]])

array([1])

In [15]:
# K Nearest Neighbours

"""
Faz classificacao com base na distancia ate algumas amostras (k) de treinamento.
A familia de algoritmos eh chamada de aprendizado baseado em instancias pois nao
ha parametros para aprender. O modelo pressupoe que a distancia eh suficiente
para fazer uma inferencia: afora isso, nenhuma pressuposicao eh feita sobre os
dados subjacentes ou suas distribuicoes.

A parte complicada eh selecionar o valor apropriado de k. Alem disso, a maldicao
da dimensionalidade pode atrapalhar as metricas de distancia, pois havera
pouca diferenca entre os vizinhos mais proximos e mais distantes nos casos de 
mais dimensoes.

"""

from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier()
knc.fit(X_train, y_train)
knc.score(X_test, y_test)

0.6528662420382165

In [16]:
knc.predict(X.iloc[[0]])

array([1])

In [18]:
# Decision Trees

"""
Uma arvore de decisao eh como ir a um medico que faz uma serie de perguntas
a fim de terminar a causa dos sintomas. Podemos usar um processo para criar uma
arvore de decisao e ter uma serie de perguntas para prever uma classe alvo.

As vantagens desse modelo incluem suporte para dados nao numericos, pouca pre-
paracao dos dados, suporte para lidar com relacionamentos nao lineares,
a importancia dos atributos eh relevada e facil de explicar.

o algoritmo padrao usado para a criacao se chama CART (Classification and 
Regression Tree). ELe usa a impureza de Gini ou medida de indices para tomada 
de decisao. Isso eh feito percorrendo os atributos em um laco e encontrando
o valor que forneca a menor probabilidade de erro de classificacao.
"""

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42, max_depth=3)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

0.7611464968152867

In [19]:
dt.predict(X.iloc[[0]])

array([1])

In [ ]:
# Random Forests

